**Классификация твитов с помощью модели RuBERT**

Есть большой датасет с твитами (файл "tweets.csv"). Нужно построить модель, которая будет определять, какие твиты негативной тональности, а какие — позитивной. Ниже приведено решение задачи классификации твитов с применением векторных представлений на базе BERT. С построением векторов текстов помогла предобученная на русских текстах модель RuBERT.

In [ ]:
# монтаж Google Диска в среде выполнения
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# импорт нужных библиотек и модулей для работы с данными
import torch
import transformers
import numpy as np
import pandas as pd
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# инициализация токенизатора
tokenizer = transformers.BertTokenizer(
    vocab_file='/content/drive/MyDrive/vocab.txt')

In [ ]:
# загрузка твитов (чтобы не создавать эмбеддинги слишком долго, берем из выборки только 400 случайных элементов)
df_tweets = pd.read_csv('/content/drive/MyDrive/tweets.csv')
df_tweets = df_tweets.sample(400).reset_index(drop=True)

In [ ]:
# токенизация текста
tokenized = df_tweets['text'].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True))

In [ ]:
# максимальная длина векторов после токенизации
print(max(len(vec) for vec in tokenized))

86


In [ ]:
# применение метода padding, чтобы после токенизации длины исходных текстов в корпусе были равными
n = max(len(vec) for vec in tokenized)
padded = np.full((400, n), 0)
for i, tweet in enumerate(tokenized):
    padded[i, :len(tweet)] = tweet
print(padded)

[[  101 15740 46032 ...     0     0     0]
 [  101  3054   846 ...     0     0     0]
 [  101   336 63205 ...     0     0     0]
 ...
 [  101 63658   303 ...     0     0     0]
 [  101  3790 22154 ...     0     0     0]
 [  101 88488   168 ...     0     0     0]]


In [ ]:
# создание маски для выделения важных токенов
attention_mask = np.where(padded != 0, 1, 0)
print(attention_mask.shape)

(400, 86)


In [ ]:
# инициализация конфигурации BertConfig и модели класса BertModel
config = transformers.BertConfig.from_json_file(
    '/content/drive/MyDrive/bert_config.json')
model = transformers.BertModel.from_pretrained(
    '/content/drive/MyDrive/rubert_model.bin', config=config)

In [ ]:
# преобразование текстов в эмбеддинги
# эмбеддинги модель BERT создаёт батчами; чтобы хватило оперативной памяти, сделаем размер батча небольшим
batch_size = 100
embeddings = []
# делаем цикл по батчам, отображать прогресс будет функция notebook()
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        # преобразование данных в формат тензоров
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

        # для ускорения вычисления функцией no_grad() укажем, что градиенты не нужны: модель BERT обучать не будем
        with torch.no_grad():
            # чтобы получить эмбеддинги для батча, передадим модели данные и маску
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        # из полученного тензора извлечём нужные элементы и добавим в список всех эмбеддингов
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# сбор всех эмбеддингов в матрицу признаков
features = np.concatenate(embeddings)

In [ ]:
# деление данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, df_tweets['positive'], test_size=0.5, random_state=42)

In [ ]:
# обучение модели логистической регрессии
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# оценка точности модели на тестовой выборке
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность на тестовой выборке: {accuracy:.4f}")

Точность на тестовой выборке: 0.9550
